In [ ]:
# Homework Week 6

# Edward Geraldo Kristian - 0706022310025
# Jevon Ivander Kangsudarmanto - 0706022310028
# Raphael Christiano Wahono - 0706022310015

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [3]:
# 1. Import libraries
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [4]:
# 2. Load the training and test datasets
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')

In [5]:
# Display the first few rows of the training data
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
# Display the first few rows of the test data
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
# 3. Data preprocessing

# Fill missing values for 'Age' with the median
train_data['Age'] = train_data['Age'].fillna(train_data['Age'].median())
test_data['Age'] = test_data['Age'].fillna(test_data['Age'].median())

# Fill missing values for 'Embarked' with the most common port
embarked_mode_train = train_data['Embarked'].mode()
if not embarked_mode_train.empty:
    train_data['Embarked'] = train_data['Embarked'].fillna(embarked_mode_train[0])

embarked_mode_test = test_data['Embarked'].mode()
if not embarked_mode_test.empty:
    test_data['Embarked'] = test_data['Embarked'].fillna(embarked_mode_test[0])

# Fill missing values for 'Fare' with the median
train_data['Fare'] = train_data['Fare'].fillna(train_data['Fare'].median())
test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].median())

# Handle missing values for 'Sex'
sex_mode_train = train_data['Sex'].mode()
if not sex_mode_train.empty:
    train_data['Sex'] = train_data['Sex'].fillna(sex_mode_train[0])

sex_mode_test = test_data['Sex'].mode()
if not sex_mode_test.empty:
    test_data['Sex'] = test_data['Sex'].fillna(sex_mode_test[0])

# Check if 'Cabin' exists before dropping
if 'Cabin' in train_data.columns:
    train_data.drop(columns=['Cabin'], inplace=True)
if 'Cabin' in test_data.columns:
    test_data.drop(columns=['Cabin'], inplace=True)

# Convert 'Sex' to numerical values
train_data['Sex'] = train_data['Sex'].map({'male': 0, 'female': 1})
test_data['Sex'] = test_data['Sex'].map({'male': 0, 'female': 1})

# Convert 'Embarked' to numerical values
train_data['Embarked'] = train_data['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})
test_data['Embarked'] = test_data['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})

# Convert all relevant columns to numeric types
numeric_columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
train_data[numeric_columns] = train_data[numeric_columns].apply(pd.to_numeric, errors='coerce')
test_data[numeric_columns] = test_data[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Check for any remaining missing or infinite values
print("Train Data Missing Values:\n", train_data.isnull().sum())
print("Test Data Missing Values:\n", test_data.isnull().sum())

# Check for infinite values
print("Train Data Infinite Values:\n", np.isinf(train_data[numeric_columns]).sum())
print("Test Data Infinite Values:\n", np.isinf(test_data[numeric_columns]).sum())

# Select features for the model including all specified columns
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X_train = train_data[features]
y_train = train_data['Survived']
X_test = test_data[features]

# Check data types
print("Train Data Types:\n", X_train.dtypes)
print("Test Data Types:\n", X_test.dtypes)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Train Data Missing Values:
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex            891
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked       891
dtype: int64
Test Data Missing Values:
 PassengerId      0
Pclass           0
Name             0
Sex            418
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked       418
dtype: int64
Train Data Infinite Values:
 Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64
Test Data Infinite Values:
 Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64
Train Data Types:
 Pclass        int64
Sex         float64
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked    float64
dtype: object
Test Data Types:
 Pclass        int64
Sex         float64
Age       

/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1047: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1052: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1072: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:87: RuntimeWarning: invalid value encountered in less_equal
  return var <= upper_bound


In [20]:
# 4. Create and Train the Logistic Regression Model

# Check for any remaining missing values in X_train
print("Missing Values in X_train:\n", pd.DataFrame(X_train).isnull().sum())

# If there are missing values, you can fill them or drop them
# For example, filling NaN values with the mean of each column
X_train = np.nan_to_num(X_train, nan=0)  # Replace NaN with 0 (or use another strategy)

# Create and train the logistic regression model
from sklearn.linear_model import LogisticRegression


# Create the logistic regression model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

Missing Values in X_train:
 0      0
1    891
2      0
3      0
4      0
5      0
6    891
dtype: int64


LogisticRegression()

In [22]:
# 5. Make Predictions

# Check for any remaining missing values in X_test
print("Missing Values in X_test:\n", pd.DataFrame(X_test).isnull().sum())

# If there are missing values, you can fill them or drop them
# For example, filling NaN values with the mean of each column
X_test = np.nan_to_num(X_test, nan=0)  # Replace NaN with 0 (or use another strategy)

# Alternatively, you can drop rows with NaN values
# X_test = X_test[~np.isnan(X_test).any(axis=1)]
# Ensure that predictions are made only on valid rows


# Make predictions on the test set
predictions = model.predict(X_test)

Missing Values in X_test:
 0      0
1    418
2      0
3      0
4      0
5      0
6    418
dtype: int64


In [23]:
# 6. Prepare the Submission File
# Create a DataFrame for submission
submission = pd.DataFrame({
    'PassengerId': test_data['PassengerId'],
    'Survived': predictions
})

# Save the submission file
submission.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
